# Job 2

Il job 2 valuta, annualmente, per ogni stato e categoria di business la valutazione media per ogni fascia di prezzo, assegnando un giudizio.

Nel dettaglio:

- per ogni business viene calcolata la valutazione media delle recensioni raggruppandole per anno;
- aggregando in base alla categoria di business, lo stato e la fascia di prezzo viene calcolata la media delle valutazioni medie delle recensioni;
- in base alla valutazione media viene elaborato un attributo aggiuntivo "business suggestion" che fornisce un giudizio sulle categorie di business, come segue:
  - valutazione media < 2: "Not recommended"
  - valutazione media 2–3.5: "Discreet"
  - valutazione media 3.5–4.5: "Recommended"
  - valutazione media > 4.5: "Highly recommended"



### Schema definitions

In [21]:
import org.apache.spark

import org.apache.spark


In [22]:
import org.apache.spark.sql.types._
import java.sql.Timestamp

val reviewSchema = StructType(
  Seq(
    StructField("user_id",  StringType,            nullable = true),
    StructField("name",     StringType,            nullable = true),
    StructField("time",     LongType,              nullable = false),
    StructField("rating",   DoubleType,            nullable = true),
    StructField("text",     StringType,            nullable = true),
    StructField("pics",     ArrayType(StringType), nullable = true),
    StructField("resp",     StructType(
      Seq(
        StructField("time", LongType,              nullable = false),
        StructField("text", StringType,            nullable = true)
      )
    ),                                             nullable = true),
    StructField("gmap_id",  StringType,            nullable = false),
  )
)

case class Response(time: Timestamp, text: Option[String])

case class Review(
  user_id: Option[String],
  name: Option[String],
  time: Timestamp,
  rating: Option[Double],
  text: Option[String],
  pics: Seq[String],
  resp: Option[Response],
  gmap_id: String
)

import org.apache.spark.sql.types._
import java.sql.Timestamp
reviewSchema: org.apache.spark.sql.types.StructType = StructType(StructField(user_id,StringType,true),StructField(name,StringType,true),StructField(time,LongType,false),StructField(rating,DoubleType,true),StructField(text,StringType,true),StructField(pics,ArrayType(StringType,true),true),StructField(resp,StructType(StructField(time,LongType,false),StructField(text,StringType,true)),true),StructField(gmap_id,StringType,false))
defined class Response
defined class Review


In [23]:
val metadataSchema = StructType(
  Seq(
    StructField("name",             StringType,                                 nullable = true),
    StructField("address",          StringType,                                 nullable = true),
    StructField("gmap_id",          StringType,                                 nullable = false),
    StructField("description",      StringType,                                 nullable = true),
    StructField("latitude",         DoubleType,                                 nullable = false),
    StructField("longitude",        DoubleType,                                 nullable = false),
    StructField("category",         ArrayType(StringType),                      nullable = true),
    StructField("avg_rating",       DoubleType,                                 nullable = false),
    StructField("num_of_reviews",   IntegerType,                                nullable = false),
    StructField("price",            StringType,                                 nullable = false),
    StructField("hours",            ArrayType(ArrayType(StringType)),           nullable = true),
    StructField("MISC",             MapType(StringType, ArrayType(StringType)), nullable = false),
    StructField("state",            StringType,                                 nullable = true),
    StructField("relative_results", ArrayType(StringType),                      nullable = true),
    StructField("url",              StringType,                                 nullable = false),
  )
)

case class Metadata(
  name: Option[String],
  address: Option[String],
  gmap_id: String,
  description: Option[String],
  latitude: Double,
  longitude: Double,
  category: Seq[String],
  avg_rating: Double,
  num_of_reviews: Int,
  price: String,
  hours: Seq[Seq[String]],
  MISC: Map[String, Seq[String]],
  state: Option[String],
  relative_results: Seq[String],
  url: String
)

metadataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(name,StringType,true),StructField(address,StringType,true),StructField(gmap_id,StringType,false),StructField(description,StringType,true),StructField(latitude,DoubleType,false),StructField(longitude,DoubleType,false),StructField(category,ArrayType(StringType,true),true),StructField(avg_rating,DoubleType,false),StructField(num_of_reviews,IntegerType,false),StructField(price,StringType,false),StructField(hours,ArrayType(ArrayType(StringType,true),true),true),StructField(MISC,MapType(StringType,ArrayType(StringType,true),true),false),StructField(state,StringType,true),StructField(relative_results,ArrayType(StringType,true),true),StructField(url,StringType,false))
defined class Metadata


### Dataset load and parse

In [24]:
import java.nio.file.Paths
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.from_unixtime

val projectDir: String = Paths.get(System.getProperty("user.dir")).getParent.getParent.getParent.toString
val reviewsPath = s"$projectDir/dataset/sample-reviews.ndjson"
val metadataPath = s"$projectDir/dataset/metadata.ndjson"

val spark = SparkSession.builder()
  .appName("NDJSON Reader")
  .master("local[*]") // Needed in local mode
  .getOrCreate()

val reviewsDf = spark.read
  .schema(reviewSchema)
  .json(reviewsPath)
  .withColumn("pics", when (col("pics") isNull, array()) otherwise col("pics"))
  .withColumn("time", from_unixtime(col("time") / 1000).cast("timestamp"))
  .withColumn("resp", 
    when (
      col("resp") isNotNull, 
      struct(
        from_unixtime(col("resp.time") / 1000).cast("timestamp").alias("time"),
        col("resp.text").cast(StringType).alias("text")
      )
    ) otherwise lit(null)
  )
  .as[Review]

val metadataDf = spark.read
  .schema(metadataSchema)
  .json(metadataPath)
  .withColumn("category", when (col("category") isNull, array()) otherwise col("category"))
  .withColumn("hours", when (col("hours") isNull, array()) otherwise col("hours"))
  .withColumn("relative_results", when (col("relative_results") isNull, array()) otherwise col("relative_results"))
  .as[Metadata]

reviewsDf.printSchema()
metadataDf.printSchema()

// Unforturnately, it seems that Spark does not support case classes in RDDs. It throws ArrayStoreException
// when trying to collect the RDD... [see also [here](https://github.com/adtech-labs/spylon-kernel/issues/40)]
val reviewsRdd = reviewsDf.rdd
  .map(Review.unapply(_).get)
  .map { case review @ (_, _, _, _, _, _, resp, _) => review.copy(_7 = resp.map(Response.unapply(_).get)) }
val metaRdd = metadataDf.rdd.map(Metadata.unapply).map(_.get)

root
 |-- user_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- rating: double (nullable = true)
 |-- text: string (nullable = true)
 |-- pics: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- resp: struct (nullable = true)
 |    |-- time: timestamp (nullable = true)
 |    |-- text: string (nullable = true)
 |-- gmap_id: string (nullable = true)

root
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- gmap_id: string (nullable = true)
 |-- description: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- category: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- avg_rating: double (nullable = true)
 |-- num_of_reviews: integer (nullable = true)
 |-- price: string (nullable = true)
 |-- hours: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: 

import java.nio.file.Paths
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.from_unixtime
projectDir: String = /Users/teo/Universita/Magistrale/BIG_DATA/bd-project25
reviewsPath: String = /Users/teo/Universita/Magistrale/BIG_DATA/bd-project25/dataset/sample-reviews.ndjson
metadataPath: String = /Users/teo/Universita/Magistrale/BIG_DATA/bd-project25/dataset/metadata.ndjson
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@7806cac3
reviewsDf: org.apache.spark.sql.Dataset[Review] = [user_id: string, name: string ... 6 more fields]
metadataDf: org.apache.spark.sql.Dataset[Metadata] = [name: string, address: string ... 13 more fields]
reviewsRdd: org.apache.spark.rdd.RDD[(Option[String], Option[String], java.sql.Timestamp, Option[Doubl...


---

**Metadata**: (name, address, <ins>gmap_id</ins>, description, latitude, longitude, category, avg_rating, num_of_reviews, price, hours, misc, state, relative_results, url)

**Review**: (user_id, name, time, rating, text, pics, responses, <ins>gmap_id</ins>)

---

In [25]:
// avg rating mapper to description
def ratingToSuggestion(rating: Double): String =
  rating match {
    case r if r <= 2.0 => "Not recommended"
    case r if r > 2.0 && r <= 3.5 => "Discreet"
    case r if r > 3.5 && r <= 4.5 => "Recommended"
    case r if r > 4.5 => "Highly recommended"
    case _ => "Undefined"
  }

def toState(address: Option[String]): String = address.flatMap { addr =>
    // This regex captures the state abbreviation between a comma and the ZIP code
    val StateRegex = """,\s*([A-Z]{2})\s+\d{5}""".r
    StateRegex.findFirstMatchIn(addr).map(_.group(1))
  }.getOrElse("Unknown")

ratingToSuggestion: (rating: Double)String
toState: (address: Option[String])String


In [26]:
val businessAvgRating = reviewsRdd
  .filter(_._4.isDefined)
  .map{ case r => 
    val rating = r._4.get
    val gmap_id = r._8
    val year = r._3.toLocalDateTime.getYear
    (gmap_id, year) -> (1, rating)
  }
  .reduceByKey((a, b) => (a._1 + b._1, a._2 + b._2)) // count reviews and sum rating value
  .map{ case ((gmap_id, year), c) => gmap_id -> (year, c._2/c._1) }

businessAvgRating: org.apache.spark.rdd.RDD[(String, (Int, Double))] = MapPartitionsRDD[60] at map at <console>:50


In [ ]:
val results = metaRdd
.filter(_._10 != null)
.flatMap(r => 
  r._7.map(category => r._3 -> (category, toState(r._2), r._10))
) //(gmap_id, (category, state, price))
.join(businessAvgRating)
.map { case (_, ((category, state, price), (year, avgRate))) => 
  (category, state, price, year) -> avgRate
}
.groupByKey()
.map{ case (key, ratings) => 
  val avgRate = ratings.sum/ratings.size
  (key, f"${avgRate}%.2f", ratingToSuggestion(avgRate))
}

<console>: 46: error: missing parameter type for expanded function ((x$1: <error>) => x$1._10)

### Save output result

In [32]:
val outputDirPath = s"$projectDir/output"
val outputPath = s"$outputDirPath/job2-output"

outputDirPath: String = /Users/teo/Universita/Magistrale/BIG_DATA/bd-project25/output
outputPath: String = /Users/teo/Universita/Magistrale/BIG_DATA/bd-project25/output/job2-output


In [33]:
import org.apache.spark.sql.SaveMode
results.map { case ((category, state, price, year), avgRate, suggestion) =>
        (category, state, price, year, avgRate, suggestion)
      }
      .coalesce(1)
      .toDF("category", "state", "price", "year", "avg_rating", "business suggestion")
      .write
      .format("csv")
      .option("header", "true")
      .mode(SaveMode.Overwrite)
      .save(s"file://$outputPath")

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 3 in stage 17.0 failed 1 times, most recent failure: Lost task 3.0 in stage 17.0 (TID 109) (192.168.0.178 executor driver): java.lang.NullPointerException: Cannot invoke "String.length()" because "$this" is null

### Plotting results

In [ ]:
%%python
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

# Carica i risultati dal file CSV generato da Spark
df = pd.read_csv("/Users/teo/Universita/Magistrale/BIG_DATA/bd-project25/output/job2-output/part-00000-e4c78d04-7c37-40b2-b4ff-95964b88d453-c000.csv")

def plot_by_year(anno_selezionato):
  df_filtrato = df[df['year'] == anno_selezionato]
  grouped = df_filtrato.groupby(['category', 'business suggestion'])['state'].nunique().reset_index()
  pivot = grouped.pivot(index='category', columns='business suggestion', values='state').fillna(0)
  pivot.plot(kind='bar', stacked=True, figsize=(12,6))
  plt.ylabel('Numero di stati')
  plt.xlabel('Categoria di business')
  plt.title(f'Numero di stati per giudizio di qualità e categoria di business ({anno_selezionato})')
  plt.legend(title='Business suggestion')
  plt.tight_layout()
  plt.show()
interact(plot_by_year, anno_selezionato=2020)
